In [1]:
from glob import glob
import rosbag
import rospy
import ros_numpy
import bagpy
import cv2
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import argparse
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError
from PIL import Image
import open3d as o3d
from ros_tools import rostools
import sys
from natsort import natsorted
sys.path.insert(0, '../optical_tracking')
from phantom_registration import manual_registration, draw_registration_result

import pandas as pd
from scipy.spatial.transform import Rotation as R
sys.path.insert(0, '../util')
from Solver import solver
sol = solver()
rt = rostools()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:

def initialization():
    global file, depth_path, zed_path, limg_path, output_dir

    output_dir = bag_file[:-4]

    valid = rt.verify_cv_bridge()
    # verify_ROS_connection()
    limg_path = output_dir +'/limg/'
    depth_path = output_dir +'/depth_pcd/'
    zed_path = output_dir +'/zed_depth_pcd/'

    if not os.path.exists(depth_path):
        os.makedirs(depth_path)
        print("Create Folder at Designated Address depth_path...")
    if not os.path.exists(zed_path):
        os.makedirs(zed_path)
        print("Create Folder at Designated Address depth_path...")
    if not os.path.exists(limg_path):
        os.makedirs(limg_path)
        print("Create Folder at Designated Address limg...")

def bag2csv(bag_file):
    b = bagpy.bagreader(bag_file)
    b.message_by_topic('/sync_pose_pan')
    b.message_by_topic('/sync_pose_camhand')
    print("Complete Saving csv.")


# def bag2pcd(bag_file, pcd_topic):
#     bridge = CvBridge()
#     start = time.time()
    
#     with rosbag.Bag(bag_file, 'r') as bag:
#         for topic, depth_msg, t in bag.read_messages(pcd_topic):
#             pcd = rt.rospc_to_o3dpc(depth_msg)
#             save_dir = os.path.join(depth_path, f'{t}.ply')
#             open3d.io.write_point_cloud(save_dir, pcd)
#             end = time.time()
#             print(end - start)
#             print("Complete Saving")

def bag2Pointcloud(bag_file, pcd_topic):
    conversion = 0.180
    depth_threshold = 0.5
    with rosbag.Bag(bag_file, 'r') as bag:
        count = 0
        start = time.time()
        for topic, depth_msg, t in bag.read_messages(pcd_topic):
            pcd = rt.rospc_to_o3dpc(depth_msg)
            R_ros2cv = np.array([[0,1,0],[0,0,-1],[-1,0,0]])  
            # R_ros2cv = np.identity(3)        
            pcd.points = o3d.utility.Vector3dVector((R_ros2cv@(np.asarray(pcd.points)*conversion).transpose(-1,-2)).transpose(-1,-2))

            # Filter out
            np_pcd= np.asarray(pcd.points)
            distances = np.sqrt(np.sum(np_pcd**2, axis=1))
            threshold = np.min(distances) + depth_threshold
            filtered_point_cloud = np_pcd[distances <= 1]
            # filtered_point_cloud = np_pcd[np.logical_and(distances <=1, distances > 0.01)]
            pcd.points = o3d.utility.Vector3dVector(filtered_point_cloud)
            # o3d.visualization.draw_geometries_with_editing([pcd])
            save_dir = os.path.join(depth_path, str(count)) + '.ply'
            o3d.io.write_point_cloud(save_dir, pcd)
            count += 1
            
        end = time.time()
        print(end - start)
        print("Complete Saving")

def bag2zedPointcloud(bag_file, zed_pcd_topic):

    with rosbag.Bag(bag_file, 'r') as bag:
        count = 0
        start = time.time()
        
        count = 0
        for topic, depth_msg, t in bag.read_messages(zed_pcd_topic):
            zed_pcd = rt.rospc_to_o3dpc(depth_msg)
            
            R_zed2cv = np.array([[0,-1,0],[0,0,-1],[1,0,0]])
            
            # pcd = o3d.geometry.PointCloud()
            zed_pcd.points = o3d.utility.Vector3dVector((R_zed2cv@np.asarray(zed_pcd.points).transpose(-1,-2)).transpose(-1,-2))

            save_dir = os.path.join(zed_path, str(count)) + '.ply'
            o3d.io.write_point_cloud(save_dir, zed_pcd)
            count += 1
            
        end = time.time()
        print(end - start)
        print("Complete Saving")

def bag2image(bag_file, l_img_topic):
    sec_list=[]
    # sec_list.append('1')
    with rosbag.Bag(bag_file, 'r') as bag:
        rt.saveImagesFromBag(bag, sec_list, l_img_topic, limg_path)

def main():
    bag2csv(bag_file)
    bag2Pointcloud(bag_file, pcd_topic)
    bag2zedPointcloud(bag_file, zed_pcd_topic)
    # bag2image(bag_file, l_img_topic)

if __name__ == '__main__':
    rt = rostools()
    bridge = CvBridge()
    global args, bag_file
    bag_file = '/media/shc/Elements/Twin-S_data/post_sync/post_sync_z.bag'
    pcd_topic = '/sync_pcd/DepthData'
    zed_pcd_topic = '/sync_zedpcd/DepthData'
    l_img_topic = '/sync_limage/compressed' 
    initialization()
    main()

AttributeError: '_geometry_msgs__PoseStamped' object has no attribute 'data'

In [46]:
import open3d as o3d
pcd_source = o3d.io.read_point_cloud('/media/shc/Elements/Twin-S_data/post_sync/post_sync_eval1/depth_pcd/280.ply')
o3d.visualization.draw_geometries_with_editing([pcd_source])

KeyboardInterrupt: 

In [33]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
            size=0.1, origin=[0, 0, 0])
pcd_source = o3d.io.read_point_cloud('/media/shc/Elements/Twin-S_data/post_sync/post_sync/cropped_1.ply')
pcd_source = o3d.io.read_point_cloud('/media/shc/Elements/Twin-S_data/post_sync/post_sync_z/zed_depth_pcd/0.ply')
# pcd_target = o3d.io.read_point_cloud('/media/shc/Elements/Twin-S_data/post_sync/post_sync/depth_pcd/0.ply')
# o3d.visualization.draw_geometries_with_editing([pcd_target])
o3d.visualization.draw_geometries([pcd_source, mesh_frame])

RPly: Unable to open file


[Open3D WARNING] Read PLY failed: unable to open file: /media/shc/Elements/Twin-S_data/post_sync/post_sync/cropped_1.ply


In [5]:
def phantom_registration(pcd_source, T_c_p_path=None):

    pcd_target = o3d.io.read_point_cloud("../data/phantom_point-cloud_data/phacon_exp_3.ply")
    pcd_target = pcd_target.voxel_down_sample(voxel_size=0.001)

    # manual registration to initialize
    init_result = manual_registration(pcd_source, pcd_target)

    threshold = 0.006
    loss = o3d.pipelines.registration.TukeyLoss(k=0.005)

    trans_init_icp = init_result.transformation

    result = o3d.pipelines.registration.registration_icp(
        pcd_source, pcd_target, threshold, trans_init_icp,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(loss))
        
    draw_registration_result(pcd_source, pcd_target, result.transformation)
    print(result)
    print(result.transformation)
    return result.transformation

idx = '90'
pcd_source = o3d.io.read_point_cloud(f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_z/depth_pcd/cropped_{idx}.ply')
T_p_ac = phantom_registration(pcd_source)
np.save(f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_z/T_p_ac_{idx}_a.npy', T_p_ac)

Visualization of two point clouds before manual alignment

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #2204 (0.051, 0.027, 0.28) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #4578 (0.026, 0.013, 0.28) to add in queue.
[Open3D INFO] Picked point #303 (0.037, 0.045, 0.29) to add in queue.


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #10116 (0.0062, 0.013, 0.00098) to add in queue.
[Open3D INFO] Picked point #10136 (-0.02, 0.0029, 0.0038) to add in queue.
[Open3D INFO] Picked point #36432 (0.0064, 0.0080, 0.025) to add in queue.

Compute a rough transform using the correspondences given by user
Perform point-to-point ICP refin

In [5]:
def getRealPose(idx, pd_data):
    df = pd_data
    pose_x = df['pose.position.x'][idx]*1000
    pose_y = df['pose.position.y'][idx]*1000
    pose_z = df['pose.position.z'][idx]*1000
    orin_x = df['pose.orientation.x'][idx]
    orin_y = df['pose.orientation.y'][idx]
    orin_z = df['pose.orientation.z'][idx]
    orin_w = df['pose.orientation.w'][idx]
    pose = np.array([pose_x, pose_y, pose_z, orin_x, orin_y, orin_z, orin_w])
    # print(real_pose, '\n', pose)
    return pose

In [6]:
date = '0411'
opti_dir = f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_z'
params_save_dir = opti_dir
T_cb_c =  np.load(f'../params/hand_eye_X_optimized_{date}.npy')
T_p_pb = np.load(f'../params/phacon2pan_{date}.npy')
T_pb_p = sol.invTransformation(T_p_pb)


pose_pan_data = os.path.join(opti_dir, 'sync_pose_pan.csv')
pose_cam_data = os.path.join(opti_dir, 'sync_pose_camhand.csv')

df_pan = pd.read_csv(pose_pan_data)
df_cam_hand = pd.read_csv(pose_cam_data)

# assert len(df_pan == df_cam_hand)
num_frames = len(df_pan)
T_c_p_batch = np.zeros([num_frames, 4, 4])
for i in range(num_frames):
    quaternion_pb = getRealPose(i, df_pan)
    quaternion_cb = getRealPose(i, df_cam_hand)
    _, T_o_pb = sol.seven2trans(quaternion_pb)
    _, T_o_cb = sol.seven2trans(quaternion_cb)
    # T_o_p_batch[i, :, :] = T_o_pb @ T_pb_p
    # T_o_cb_batch[i,:,:] = T_o_cb
    T_o_c = T_o_cb@T_cb_c
    # offset 
    T_c_corr = np.identity(4)
    T_c_corr[0,3] = 8.3
    T_c_corr[1,3] = 5.6 
    T_o_c = np.dot(T_o_c, T_c_corr)

    T_o_p = T_o_pb @ T_pb_p
    T_c_p_real = sol.invTransformation(T_o_c)@T_o_p
    T_c_p_batch[i,:,:] = T_c_p_real
    
    np.save(os.path.join(params_save_dir, f'T_c_p_real_offset.npy'),T_c_p_batch)
    # print(T_c_p_real[:3, 3])
    # break
# np.save(os.path.join(params_save_dir, f'T_o_p_opti{opti_num}_0321.npy'), T_o_p_batch)
# np.save(os.path.join(params_save_dir, f'T_o_cb_opti{opti_num}_0321.npy'), T_o_cb_batch)


Visualize ZED, AMBF, and CT

In [16]:
date = '0411'
opti_dir = f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_z'
T_c_p = np.load(os.path.join(opti_dir, f'T_c_p_real_offset.npy'))
zed_pcd_dir = os.path.join(opti_dir, 'zed_depth_pcd')
ambf_pcd_dir = os.path.join(opti_dir, 'depth_pcd')
T_c_p[...,:3 ,3] /= 1000
pcd_target = o3d.io.read_point_cloud("../data/phantom_point-cloud_data/phacon_exp_3.ply")

zed_pointclouds = natsorted([os.path.join(zed_pcd_dir, f) for f in os.listdir(zed_pcd_dir) if ".ply" in f])
ambf_pointclouds = natsorted([os.path.join(ambf_pcd_dir, f) for f in os.listdir(ambf_pcd_dir) if ".ply" in f])

# visulize result
for [count,pointcloud] in enumerate(zip(zed_pointclouds, ambf_pointclouds)):
    pcd_zed = o3d.io.read_point_cloud(os.path.join(zed_pcd_dir, pointcloud[0]))
    pcd_ambf = o3d.io.read_point_cloud(os.path.join(ambf_pcd_dir, pointcloud[1]))
    pcd_ambf.paint_uniform_color([1, 0.706, 1])
    # draw_registration_result(pcd_target, pcd_zed, T_c_p)
    
    pcd_target.transform(T_c_p[count])
    o3d.visualization.draw_geometries([pcd_target, pcd_zed, pcd_ambf], front=[ -0.3509939981295529, -0.0013414501852417977, -0.93637674778287405 ],
        lookat=[ -0.0044998621935288937, -0.029191735052132683, 0.34626969113492007 ],
        up=[ 0.90668866085024358, 0.24932730419749116, -0.34022282061496678 ],
        zoom=0.38000000000000006)
    pcd_target.transform(sol.invTransformation(T_c_p[count]))

KeyboardInterrupt: 

Visualize ZED and CT

In [49]:
# date = '0411'
opti_dir = f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_eval2'
T_c_p = np.load(os.path.join(opti_dir, f'T_c_p_real.npy'))
ambf_pcd_dir = os.path.join(opti_dir, 'depth_pcd')
T_c_p[...,:3 ,3] /= 1000
pcd_target = o3d.io.read_point_cloud("../data/phantom_point-cloud_data/phacon_exp_3.ply")

ambf_pointclouds = natsorted([os.path.join(ambf_pcd_dir, f) for f in os.listdir(ambf_pcd_dir) if ".ply" in f])

# visulize result
for [count,pointcloud] in enumerate(ambf_pointclouds):
    pcd_ambf = o3d.io.read_point_cloud(os.path.join(ambf_pcd_dir, pointcloud))
    pcd_ambf.paint_uniform_color([1, 0.706, 1])
    
    pcd_target.transform(T_c_p[count])
    o3d.visualization.draw_geometries([pcd_target, pcd_ambf], front=[ -0.14054829969218396, -0.21430914598217043, -0.96660114080319037 ],
        lookat=[ -0.064506968418826249, -0.017665959520745674, 0.34775550426918295 ],
        up=[ 0.95587178555563934, -0.28376383648487885, -0.076073745024526532 ],
        zoom=0.46)
    pcd_target.transform(sol.invTransformation(T_c_p[count]))

KeyboardInterrupt: 

Eval overlay of pointcloud and RGB of ZED

In [2]:
import cv2
import numpy as np
import matplotlib as mpl
def project_pointcloud_to_image(pointcloud, intrinsics, extrinsics, image_shape):
    # Convert point cloud from 3D coordinates to 2D image coordinates
    points_2d, _ = cv2.projectPoints(pointcloud, extrinsics[:3], extrinsics[3:], intrinsics, distCoeffs=None)
    # Scale the 2D points to the size of the image
    points_2d = np.squeeze(points_2d)
    points_2d[:, 0] = np.clip(points_2d[:, 0], 0, image_shape[1] - 1)
    points_2d[:, 1] = np.clip(points_2d[:, 1], 0, image_shape[0] - 1)
    points_2d = np.round(points_2d).astype(int)

    # Create an empty image
    image = np.zeros((image_shape[0], image_shape[1], 3), dtype=np.uint8)
    dists = np.linalg.norm(pointcloud[:, :3], axis=1)  # compute distances from point to camera

    # # Assign RGB colors to the projected points
    cmap_norm = mpl.colors.Normalize(vmin=np.min(dists), vmax=np.max(dists))
    pixs_colors = plt.get_cmap('jet')(cmap_norm(dists))[:, 0:3]*255
    for i, point in enumerate(points_2d):
        c_ = tuple(pixs_colors[i].astype(np.uint8))
        c_ = np.uint8(c_)
        x, y = point
        r, g, b = int(c_[0]),int(c_[1]),int(c_[2])
        image[y, x] = [r, g, b]
    return image

In [4]:
opti_dir = f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_z'
zed_pcd_dir = os.path.join(opti_dir, 'zed_depth_pcd')
intrinsics = np.load('../params/zed_M_l.npy')
extrinsics = np.zeros([6,1])
image_shape = (1080, 1920)
zed_pointclouds = natsorted([os.path.join(zed_pcd_dir, f) for f in os.listdir(zed_pcd_dir) if ".ply" in f])

output_file = cv2.VideoWriter(
            filename='/media/shc/Elements/Twin-S_data/post_sync/post_sync_z/zed_overlay.mp4',
            fourcc=cv2.VideoWriter_fourcc(*"mp4v"),
            fps=15,
            frameSize=(1920, 1080),
            isColor=True,
        )

for [count,zed_pointcloud] in enumerate(zed_pointclouds):
    zed_pcd = o3d.io.read_point_cloud(zed_pointcloud)
    zed_pcd = np.asarray(zed_pcd.points)
    print(zed_pcd.shape)
    pcd_img = project_pointcloud_to_image(zed_pcd, intrinsics, extrinsics, image_shape)

    left_img = cv2.imread(f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_z/limg/{count}.jpeg')
    img = cv2.addWeighted(left_img, 0.5, pcd_img, 0.9, 0)
    # cv2.imwrite(f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_z/zed_overlay/{count}.jpeg', img)
    output_file.write(img)
    # if count == 50:
    #     break
output_file.release()
    # break

(85628, 3)
(92322, 3)
(88464, 3)


KeyboardInterrupt: 

ambf overlay

In [29]:
opti_dir = f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_z'
ambf_pcd_dir = os.path.join(opti_dir, 'depth_pcd')
intrinsics = np.load('../params/zed_M_l.npy')
intrinsics[0, 2] = 960
intrinsics[1, 2] = 540
print(intrinsics)

extrinsics = np.zeros([6,1])

image_shape = (1080, 1920)
ambf_pointclouds = natsorted([os.path.join(ambf_pcd_dir, f) for f in os.listdir(ambf_pcd_dir) if ".ply" in f])

output_file = cv2.VideoWriter(
            filename='/media/shc/Elements/Twin-S_data/post_sync/post_sync_z/ambf_overlay_.mp4',
            fourcc=cv2.VideoWriter_fourcc(*"mp4v"),
            fps=15,
            frameSize=(1920, 1080),
            isColor=True,
        )

for [count,ambf_pointcloud] in enumerate(ambf_pointclouds):
    ambf_pcd = o3d.io.read_point_cloud(ambf_pointcloud)
    ambf_pcd = np.asarray(ambf_pcd.points)
    pcd_img = project_pointcloud_to_image(ambf_pcd, intrinsics, extrinsics, image_shape)
    sim_img = cv2.imread(f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_z/sim_img/{count}.jpeg')
    sim_img = cv2.resize(sim_img, (1920, 1080))
    left_img = cv2.imread(f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_z/limg/{count}.jpeg')
    img = cv2.addWeighted(sim_img, 0.5, pcd_img, 0.9, 0)
    # cv2.imwrite(f'/media/shc/Elements/Twin-S_data/post_sync/post_sync_z/zed_overlay/{count}.jpeg', img)
    output_file.write(img)
    # if count == 50:
    #     break
output_file.release()
    # break

[[1.44665466e+03 0.00000000e+00 3.20000000e+02]
 [0.00000000e+00 1.44665466e+03 1.80000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
